In [2]:
import os
import re
import json
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModel
import warnings

In [3]:
def data_import(data_name):
    data_path = os.getenv('HOME')+'/aiffel/project_data/dlthon/'+data_name
    imported_data = pd.read_csv(data_path)
    return imported_data

def cleaning_sentence(sentence):
        sentence = sentence.lower()
        sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
        sentence = re.sub(r'\([^)]*\)', '', sentence)
        sentence = re.sub(r'[" "]+', " ", sentence)
        sentence = re.sub("[^가-힣a-zA-Z0-9\.\?\!,]+", " ", sentence)
        sentence = re.sub(r'[\n\r]+', ' ', sentence)
        sentence = sentence.strip()
        return sentence

def preprocess_sentence(data_list):
    retrun_list = []
    for sentence_frame in data_list:
         befor_df = {}
         conv_data = []
         class_data = []
         class_name = sentence_frame['class'][0]
         for sentence in sentence_frame['conversation']:
             cleaned_sentence = cleaning_sentence(sentence)
             conv_data.append(cleaned_sentence)
             class_data.append(class_name)
         return_df = pd.DataFrame({'class' : class_data, 'conversation': conv_data})
         retrun_list.append(return_df)
    return retrun_list

def random_deletion(text, prob=0.2):
    words = text.split()
    if len(words) == 1:
        return text
    return ' '.join([word for word in words if random.random() > prob])

def random_swap(text, n=1):
    words = text.split()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

def convert_label(data):
    label_map = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
    
    data['class'] = data['class'].map(label_map)
    return data

def cal_len(train, test, rate):
    con_data = np.concatenate((train, test), axis = 0)
    seg_len = []
    spl_len = []
    for i in con_data:
        single_seg_len = len(i)
        seg_len.append(single_seg_len)
    for i in con_data:
        single_spl_len = len(i.split())
        spl_len.append(single_spl_len)
    spl_len.sort()
    seg_len.sort()
    print('spl len is : ', spl_len[int(len(spl_len)*rate)])
    print('seg len is : ', seg_len[int(len(seg_len)*rate)])

In [4]:
def data_aug(data_list, prob, n):
    len_data = []

    for data in data_list:
        len_data.append(len(data))

    max_len_of = max(len_data)

    return_data = []

    for data_set in data_list:
        if len(data_set) != max_len_of:
            conver_data = []
            class_Data = []
            return_df = {}
            aug_len = max_len_of - len(data_set)
            class_name = data_set['class'][0]
            for i in range(aug_len): class_Data.append(class_name)
            for i in range(aug_len):
                choice_num = random.random()
                random_seq = data_set['conversation'].sample(1).iloc[0]
                if choice_num >= 0.5:
                    output_seq = random_deletion(random_seq, prob)
                    conver_data.append(output_seq)
                else:
                    output_seq = random_swap(random_seq, n)
                    conver_data.append(output_seq)
                    
            retrun_df = pd.DataFrame({'class':class_Data, 'conversation':conver_data})
        else:
            retrun_df = 0

        return_data.append(retrun_df)

    final_list = []

    for auged_data, real_data in zip(return_data, data_list):
        if isinstance(auged_data, pd.DataFrame):
            real_data = pd.concat([real_data, auged_data])
            real_data.reset_index(drop=True, inplace=True)
            final_list.append(real_data)
        else:
            final_list.append(real_data)
    
    return final_list

In [5]:
bert_model = TFAutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [6]:
def bert_encode(datas, sent_max_length):
    input_ids = []
    attention_masks = []
    
    for sent in datas:
        encoded = tokenizer.encode_plus(sent,
                                        add_special_tokens = True,
                                        max_length = sent_max_length,
                                        padding='max_length',
                                        truncation = True,
                                        return_attention_mask=True)
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids), np.array(attention_masks)

In [7]:
def create_model(bert_model, max_len):
    input_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32)
    
    output = bert_model([input_ids, attention_mask])
    output = output.last_hidden_state
    output = tf.keras.layers.Dense(64, activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(3, activation='softmax')(output)
    
    model = tf.keras.Model(inputs = [input_ids, attention_mask], outputs = output)
    model.compile(Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
train_data = data_import('train_arg.csv')
train_data = train_data.drop(['data_type', 'idx'], axis=1)
nomal_data = data_import('preprocessed_train_raw.csv')
nomal_data = nomal_data[nomal_data['class'] == 4]
nomal_data['class'] = nomal_data['class'].map({4 : '일반 대화'})
nomal_data = nomal_data.sample(20000)
nomal_data = nomal_data.drop('Unnamed: 0', axis=1)
nomal_data = nomal_data[['class', 'conversation']]
all_data = pd.concat([train_data, nomal_data])
all_data = all_data.sample(frac=1).reset_index(drop=True)

In [15]:
proto_data = np.array(all_data['conversation'])

In [16]:
changed_data = []

for sentence in proto_data:
    sentence = re.sub(r'\n', ' 옌 ', sentence)
    changed_data.append(sentence)

prot_data = np.array(changed_data)

In [17]:
prot_data

array(['믿었던 사람이 나를 버렸어. 아무도 만나고 싶지 않아. 옌 믿음이 컸던 사람이 떠나서 더 마음이 아프시겠고 사람을 쉽게 믿지 못하시겠어요. 옌 믿음이 컸었는데 한순간에 떠나버렸어. 앞으로는 그 누구와도 사귀지 않을 예정이야. 옌 어떻게 하면 지금의 감정을 완화시킬 수 있을까요? 옌 당분간 혼자 있으면서 생각도 정리하고 마음의 안식을 갖는 시간을 가져 볼까 해. 옌 혼자만의 시간이 의미 있는 시간으로 받아들여졌으면 좋겠어요.',
       '치매에 걸려 딸을 잘 알아보지 못하는 나 자신이 너무 싫어. 옌 몸이 안 좋으신 일로 자기비난을 하시는 거군요. 옌 길을 찾지 못해서 밖을 나가지도 못해. 나 내 정신 탓이야. 옌 자기 비난에 빠진 감정을 어떻게하면 바꿀 수 있을까요? 옌 병을 최대한 치료하고 늦출 수 있게 노력해야겠지. 옌 몸조리를 잘해서 최대한 진행을 늦출 수 있었으면 좋겠어요.',
       '움직이기가 너무 귀찮아 운동을 좀 해야 하는데. 옌 요즘 하시는 운동이 있으세요? 옌 걷기라도 해야 하는데 꼼짝하기가 싫어. 옌 운동에 대한 생각은 있으신데 성가심을 느끼고 계시군요.  옌 맞아. 그렇지만 오늘은 저녁 먹고 슬슬 나가봐야겠어. 옌 막상 나가시면 뿌듯하실 거예요.',
       ...,
       '저기요. 쿠팡잇츠에서 시킨 사람인데요. 빙수 다른데요? 옌 네? 혹시 주문번호가 어떻게 되나요.? 옌 203번이요 옌 아. 사인머스켓 시켜셨지요? 문제인가요? 옌 네. 샤인머스켓에서 단맛이 없고 사과처럼 퍼석 옌 그럴리가 다른 고객님들은 아직 컴플레인 혹시 불 옆에서 하시거나 다른 음식이랑 아니신가요? 옌 팥빙수를 그냥 먹지 조리를 어떻게 미친거아니야. 그리고 내가 다른 음식이랑 왜 섞어서 먹어요. 됐고 빨리 환불해주세요. 옌 분 음식이 어떻게 할까요? 옌 또 우리만 손해니까 그냥 환불해드려. 옌 네. 알겠습니다. 고객님 들여 지금 환불해드리겠습니다.',
       '아이들을 가르치는 일이 좋아서 시작했는데 이젠 교탁에 서 있기도 

In [18]:
train_data_a = np.array(prot_data[:int(0.8*len(prot_data))], dtype = str)
test_data_a = np.array(prot_data[int(0.8*len(prot_data)):], dtype = str)

In [19]:
cal_len(train_data_a, test_data_a, 0.999999)

spl len is :  240
seg len is :  896


In [20]:
train_input_ids_a, train_attention_mask_a = bert_encode(train_data_a, 240)

In [21]:
print(tokenizer.encode('옌'))

[0, 6, 246458, 2]


In [22]:
train_input_ids_a[1]

array([     0,  34542,  16595,    480,  29072,   9178, 118100,    413,
        11817, 153484,   1190, 167003,   8000, 136214,  31369, 233604,
            5,      6, 246458,  43972,    469,   6888,      6,  87648,
        28644,   5301,   6571,   1083,  57049,   4274,  22738,    413,
            6, 106343,  11470, 160930,      5,      6, 246458, 175926,
            6,  77925,   1190,  22058,  23069,      6, 164540,    413,
       121977,  57953,  22058,   1963,      5,   8000,   5920,  97053,
            6, 239357,    469,   6838,      5,      6, 246458,  57049,
         7091,  22738,    480,  89392,   4033, 143987,    413,  35078,
        17801,  11095, 170801,   1020, 154325,   5144,     32,      6,
       246458,  80332,    413, 207145,  70439,   2382,      6, 249950,
        14842,   1020,  92234,  54122,  19353,  54232,   1190,      5,
            6, 246458,  43972,   6740,  80860,  11817,  23069, 207145,
        19217,    413,      6, 249950,  14842,   1020,  72121,  37557,
      

In [23]:
def make_laber(data, index_num, max_len):
    return_data = []
    
    for conv in data:
        config_num = 0
        return_sigle_sent = []
        for sen in conv:
            if sen == index_num:
                if config_num == 0:
                    config_num = 1
                else:
                    config_num = 0
            elif sen == 1:
                return_sigle_sent.append(2)
            else :
                return_sigle_sent.append(config_num)
        return_sigle_sent = np.array(return_sigle_sent)
        if return_sigle_sent.shape[0] != max_len:
            add_len = max_len - return_sigle_sent.shape[0]
            num_in = return_sigle_sent[-1]
            for i in range(add_len):
                return_sigle_sent = np.append(return_sigle_sent, int(num_in))
            return_sigle_sent = np.reshape(return_sigle_sent, (1, max_len))
        return_data.append(return_sigle_sent)
    
    return return_data        

In [24]:
train_label_ids = make_laber(train_input_ids_a,246458, 240)

In [25]:
train_label_ids_np = np.concatenate(train_label_ids, axis=0)

In [26]:
train_label_ids_np[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [27]:
changed_data = []

for sentence in proto_data:
    sentence = re.sub(r'[\n\r]+', ' ', sentence)
    changed_data.append(sentence)

proto_data = np.array(changed_data)

In [28]:
train_data_b = np.array(proto_data[:int(0.8*len(proto_data))], dtype = str)
test_data_b = np.array(proto_data[int(0.8*len(proto_data)):], dtype = str)

In [29]:
train_a_input_ids_b, train_a_attention_mask_b = bert_encode(train_data_b, 240)

In [30]:
train_a_input_ids_b[1]

array([     0,  34542,  16595,    480,  29072,   9178, 118100,    413,
        11817, 153484,   1190, 167003,   8000, 136214,  31369, 233604,
            5,  43972,    469,   6888,      6,  87648,  28644,   5301,
         6571,   1083,  57049,   4274,  22738,    413,      6, 106343,
        11470, 160930,      5, 175926,      6,  77925,   1190,  22058,
        23069,      6, 164540,    413, 121977,  57953,  22058,   1963,
            5,   8000,   5920,  97053,      6, 239357,    469,   6838,
            5,  57049,   7091,  22738,    480,  89392,   4033, 143987,
          413,  35078,  17801,  11095, 170801,   1020, 154325,   5144,
           32,  80332,    413, 207145,  70439,   2382,      6, 249950,
        14842,   1020,  92234,  54122,  19353,  54232,   1190,      5,
        43972,   6740,  80860,  11817,  23069, 207145,  19217,    413,
            6, 249950,  14842,   1020,  72121,  37557, 132198,  38736,
            5,      2,      1,      1,      1,      1,      1,      1,
      

In [31]:
train_a_attention_mask_b[1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [32]:
use_bert_model = create_model(bert_model, 240)
use_bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 240)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 117653760   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 240, 64)      24640       tf_bert_model[0][13]         

In [33]:
tf.keras.backend.clear_session()

In [34]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("dlthon2.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max', save_weights_only=True)

In [35]:
history = use_bert_model.fit([train_a_input_ids_b, train_a_attention_mask_b], train_label_ids_np, validation_split=0.2, epochs = 10, callbacks=[checkpoint], batch_size=8)

Epoch 1/10
2547/2547 [==============================] - 642s 246ms/step - loss: 0.1692 - accuracy: 0.9091 - val_loss: 0.1279 - val_accuracy: 0.9328

Epoch 00001: val_accuracy improved from -inf to 0.93280, saving model to dlthon2.keras
Epoch 2/10
2547/2547 [==============================] - 625s 246ms/step - loss: 0.1196 - accuracy: 0.9399 - val_loss: 0.1197 - val_accuracy: 0.9400

Epoch 00002: val_accuracy improved from 0.93280 to 0.94005, saving model to dlthon2.keras
Epoch 3/10
2547/2547 [==============================] - 626s 246ms/step - loss: 0.1038 - accuracy: 0.9493 - val_loss: 0.1116 - val_accuracy: 0.9442

Epoch 00003: val_accuracy improved from 0.94005 to 0.94417, saving model to dlthon2.keras
Epoch 4/10
2547/2547 [==============================] - 626s 246ms/step - loss: 0.0898 - accuracy: 0.9571 - val_loss: 0.1083 - val_accuracy: 0.9510

Epoch 00004: val_accuracy improved from 0.94417 to 0.95098, saving model to dlthon2.keras
Epoch 5/10
2547/2547 [=========================

In [36]:
test_data_a

array(['상반기 첫 성과급이 예상보다 더 많이 들어왔어. 예상치 못한 기쁨이야! 옌 성과급을 예상보다 많이 받으셔서 기쁘시군요. 얼마나 많이 받으셨나요? 옌 예상보다 백만 원은 더 들어온 것 같아. 너무 행복해! 옌 성과급을 어디에 사용하고 싶으신가요? 옌 부모님께 선물을 사드릴 생각이야. 정말 기뻐하실 거야. 옌 부모님께 선물을 사드릴 생각에 기쁘시군요.',
       '어제 갑자기 큰 아이가 자동차를 사야겠단 이야기를 하더군. 아무래도 직장을 다니면서 대중교통을 이용하는 것이 불편한가 봐. 옌 그런 일이 있으셨군요. 어떻게 하실 생각이세요? 옌 집사람과 의논도 해보았지만 답이 안 나와. 괜히 차를 몰고 다니면 또 직장동료들이나 친구들 사이에서 기사 역할을 도맡게 되는 것이 아닌가 싶기도 하고. 옌 그러시군요. 왜 그런 생각이 드시는 건가요? 옌 우리 아이가 거절을 잘 못하는 스타일이라서 그래. 괜히 기사 역할 하게 될까 봐 걱정이 많이 드는 것은 사실이야. 차를 사주어야 할지 말아야 할지 참 헷갈리는군. 옌 걱정이 많이 되시겠어요. 그런 점이 걱정되셔서 쉽게 차를 사주시지 못하시겠어요.',
       '너 일로 와봐. 옌 네. 옌 너희 아까 손님들 앞에서 너희끼리 떠들면서 웃었니? 옌 네??.아니요 아까 손님께서 말 거셔서 그냥 웃은 거에요. 옌 여기가 어디라고 웃어 너희 일하러 왔지 놀러왔니? 옌 죄송합니다. 옌 하여튼 가봐.일이나 제대로 하면 말도 안해 옌 네 그럼 가 보겠습니다. 옌 아참가서 식빵 4개 모닝롤 3개 단호박빵 3개 가져와. 옌 매니저님 죄송한데 다시 한번만 말씀해 주실 수 있으실까요? 옌 하.됐어 내가 할라니까 옌 죄송합니다.',
       ...,
       '저기요. 쿠팡잇츠에서 시킨 사람인데요. 빙수 다른데요? 옌 네? 혹시 주문번호가 어떻게 되나요.? 옌 203번이요 옌 아. 사인머스켓 시켜셨지요? 문제인가요? 옌 네. 샤인머스켓에서 단맛이 없고 사과처럼 퍼석 옌 그럴리가 다른 고객님들은 아직 컴플레인 혹시 불 옆에서

In [37]:
test_a_input_ids, test_a_attention_mask = bert_encode(test_data_a, 240)

In [38]:
test_label_ids = make_laber(test_a_input_ids,246458, 240)

In [39]:
test_label_ids_np = np.concatenate(test_label_ids, axis=0)

In [40]:
test_a_input_ids_b, test_a_attention_mask_b = bert_encode(test_data_b, 240)

In [41]:
use_bert_model.evaluate([test_a_input_ids_b, test_a_attention_mask_b], test_label_ids_np)

199/199 [==============================] - 42s 213ms/step - loss: 0.0969 - accuracy: 0.9630


[0.09694472700357437, 0.9629745483398438]

In [42]:
test_a_pridict = use_bert_model.predict([test_a_input_ids_b, test_a_attention_mask_b])

In [44]:
test_a_predictions = test_a_pridict.argmax(axis=-1)

In [51]:
test_a_predictions[1000]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [52]:
test_label_ids_np[1000]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [55]:
test_data_b[1000]

'이번 시험 점수가 떨어지지 않아서 다행이야. 시험 점수가 떨어지지 않으셨군요. 이번 시험을 위해 무슨 노력을 하셨나요? 수업 시간에 집중해서 공부했어. 내가 원하는 학과를 못 갈까 봐 열심히 했어. 노력한 만큼 좋은 결과가 나와서 행복하시겠어요.'